In [ ]:
!pip install python-chess tensorflow
!pip show keras tensorflow h5py

In [ ]:
import json
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import load_model
import chess

with open("master_games_with_color.jsonl", "r") as f:
    games = [json.loads(line.strip()) for line in f]

all_moves = set()
for game in games:
    for move in game.get("moves", []):
        if isinstance(move, dict) and "move" in move and "color" in move:
            combo = f"{move['color']}_{move['move']}"
            all_moves.add(combo)

move_to_id = {move: idx + 1 for idx, move in enumerate(sorted(all_moves))}
id_to_move = {idx: move for move, idx in move_to_id.items()}

with open("move_to_id.pkl", "wb") as f:
    pickle.dump(move_to_id, f)

def convert_moves_to_ids(moves):
    return [
        move_to_id[f"{move['color']}_{move['move']}"]
        for move in moves
        if isinstance(move, dict) and "move" in move and "color" in move and f"{move['color']}_{move['move']}" in move_to_id
    ]

sequence_length = 5
games_as_ids = [
    game_ids for game_ids in [convert_moves_to_ids(game.get("moves", [])) for game in games]
    if len(game_ids) > sequence_length
]

X, y = [], []
for game in games_as_ids:
    for i in range(len(game) - sequence_length):
        X.append(game[i:i + sequence_length])
        y.append(game[i + sequence_length])

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = Sequential()
model.add(Embedding(input_dim=len(move_to_id) + 1, output_dim=64, input_length=sequence_length))
model.add(LSTM(128))
model.add(Dense(len(move_to_id) + 1, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

model.save("chess_move_predictor_model.keras")

model = load_model("chess_move_predictor_model.keras")

board = chess.Board()
moves_san = ["e4", "e5", "Nf3", "Nc6", "Bc4"]

for move in moves_san:
    try:
        board.push_san(move)
    except ValueError:
        print(f"Geçersiz hamle: {move}")

last_moves = moves_san[-sequence_length:]
last_moves_ids = []

for san_move, color in zip(last_moves, ["white", "black"] * (len(last_moves) // 2 + 1)):
    move_str = f"{color}_{san_move}"
    if move_str in move_to_id:
        last_moves_ids.append(move_to_id[move_str])

# === 16. En olası 5 hamleyi tahmin et ===
input_seq = np.array([last_moves_ids])
predictions = model.predict(input_seq)[0]
top_5_indices = predictions.argsort()[-5:][::-1]
top_5_moves = [id_to_move[idx] for idx in top_5_indices]

print("\nTahmin edilen en olası 5 hamle:")
for i, move in enumerate(top_5_moves, 1):
    print(f"{i}. {move}")

played = False
for move_str in top_5_moves:
    try:
        _, san = move_str.split("_", 1)
        move = board.parse_san(san)
        if move in board.legal_moves:
            board.push(move)
            print(f"\n✅ '{san}' hamlesi legal ve oynandı.")
            played = True
            break
    except Exception as e:
        continue

if not played:
    print("⚠️ Tahmin edilen hamleler arasında legal bir hamle bulunamadı.")
